<a href="https://colab.research.google.com/github/realgjl/Polars-guide/blob/main/polars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import polars as pl

create some polars dataframe

In [2]:
data = {'employee_id': [1, 1, 1, 2, 2, 2],
        'datetime_start': ['2023-01-02 00:00:00', '2023-01-02 05:00:00', '2023-01-02 06:00:00',
                  '2023-01-01 23:00:00', '2023-01-02 02:00:00', '2023-01-02 08:00:00'],
        'datetime_end': ['2023-01-02 05:00:00', '2023-01-02 06:00:00', '2023-01-02 11:00:00',
                '2023-01-02 02:00:00', '2023-01-02 08:00:00', '2023-01-02 13:00:00'],
        'type': ['working', 'resting', 'working', 'working', 'resting', 'working'],
        'shift_id': [1, 2, 3, 1, 2, 3]}
timesheet_df = pl.DataFrame(data)
timesheet_df

employee_id,datetime_start,datetime_end,type,shift_id
i64,str,str,str,i64
1,"""2023-01-02 00:…","""2023-01-02 05:…","""working""",1
1,"""2023-01-02 05:…","""2023-01-02 06:…","""resting""",2
1,"""2023-01-02 06:…","""2023-01-02 11:…","""working""",3
2,"""2023-01-01 23:…","""2023-01-02 02:…","""working""",1
2,"""2023-01-02 02:…","""2023-01-02 08:…","""resting""",2
2,"""2023-01-02 08:…","""2023-01-02 13:…","""working""",3


sometimes need to transfer a string to polars dattime format

In [3]:
def polars_to_datetime(datetimeLikeString):

    test_df = pl.DataFrame({'date_str': [datetimeLikeString]})
    date_df = test_df.with_columns(pl.col('date_str').str.strptime(pl.Datetime, format='%Y-%m-%d %H:%M:%S').cast(pl.Datetime))
    returned_datetime = list(date_df[0])[0][0]

    return returned_datetime


job_start = polars_to_datetime('2023-01-02 12:00:00')
job_end = polars_to_datetime('2023-01-02 13:00:00')

print(job_start)

2023-01-02 12:00:00


convert inside the csv as well

In [4]:
timesheet_df = timesheet_df.with_columns(
    pl.col('datetime_start')
    .str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False)
    .alias('datetime_start')
)
timesheet_df = timesheet_df.with_columns(
    pl.col('datetime_end')
    .str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S", strict=False)
    .alias('datetime_end')
)
print(timesheet_df)

shape: (6, 5)
┌─────────────┬─────────────────────┬─────────────────────┬─────────┬──────────┐
│ employee_id ┆ datetime_start      ┆ datetime_end        ┆ type    ┆ shift_id │
│ ---         ┆ ---                 ┆ ---                 ┆ ---     ┆ ---      │
│ i64         ┆ datetime[μs]        ┆ datetime[μs]        ┆ str     ┆ i64      │
╞═════════════╪═════════════════════╪═════════════════════╪═════════╪══════════╡
│ 1           ┆ 2023-01-02 00:00:00 ┆ 2023-01-02 05:00:00 ┆ working ┆ 1        │
│ 1           ┆ 2023-01-02 05:00:00 ┆ 2023-01-02 06:00:00 ┆ resting ┆ 2        │
│ 1           ┆ 2023-01-02 06:00:00 ┆ 2023-01-02 11:00:00 ┆ working ┆ 3        │
│ 2           ┆ 2023-01-01 23:00:00 ┆ 2023-01-02 02:00:00 ┆ working ┆ 1        │
│ 2           ┆ 2023-01-02 02:00:00 ┆ 2023-01-02 08:00:00 ┆ resting ┆ 2        │
│ 2           ┆ 2023-01-02 08:00:00 ┆ 2023-01-02 13:00:00 ┆ working ┆ 3        │
└─────────────┴─────────────────────┴─────────────────────┴─────────┴──────────┘


👆 you can see the formate of datetime_start and datetime_end is different now.



filter is so easy here!

In [5]:
# %%timeit

employee_ids = [1]
# condition: id is 1 and type is not resting
condition = (pl.col('employee_id').is_in(employee_ids)) & (~(pl.col('type') == 'resting'))
result = timesheet_df.filter(condition)
print(result)

shape: (2, 5)
┌─────────────┬─────────────────────┬─────────────────────┬─────────┬──────────┐
│ employee_id ┆ datetime_start      ┆ datetime_end        ┆ type    ┆ shift_id │
│ ---         ┆ ---                 ┆ ---                 ┆ ---     ┆ ---      │
│ i64         ┆ datetime[μs]        ┆ datetime[μs]        ┆ str     ┆ i64      │
╞═════════════╪═════════════════════╪═════════════════════╪═════════╪══════════╡
│ 1           ┆ 2023-01-02 00:00:00 ┆ 2023-01-02 05:00:00 ┆ working ┆ 1        │
│ 1           ┆ 2023-01-02 06:00:00 ┆ 2023-01-02 11:00:00 ┆ working ┆ 3        │
└─────────────┴─────────────────────┴─────────────────────┴─────────┴──────────┘


In [6]:
print(result)

shape: (2, 5)
┌─────────────┬─────────────────────┬─────────────────────┬─────────┬──────────┐
│ employee_id ┆ datetime_start      ┆ datetime_end        ┆ type    ┆ shift_id │
│ ---         ┆ ---                 ┆ ---                 ┆ ---     ┆ ---      │
│ i64         ┆ datetime[μs]        ┆ datetime[μs]        ┆ str     ┆ i64      │
╞═════════════╪═════════════════════╪═════════════════════╪═════════╪══════════╡
│ 1           ┆ 2023-01-02 00:00:00 ┆ 2023-01-02 05:00:00 ┆ working ┆ 1        │
│ 1           ┆ 2023-01-02 06:00:00 ┆ 2023-01-02 11:00:00 ┆ working ┆ 3        │
└─────────────┴─────────────────────┴─────────────────────┴─────────┴──────────┘


sometimes need a list of one column of the results

In [7]:
result_id_list = timesheet_df.filter(condition)["employee_id"].to_list()
print(result_id_list)

[1, 1]


add another column based on the given data (do some math)

In [8]:
result = result.with_columns([
    (pl.col('datetime_end') - pl.col('datetime_start')).dt.hours().alias('duration (hrs)')
])
print(result)

shape: (2, 6)
┌─────────────┬─────────────────────┬─────────────────────┬─────────┬──────────┬────────────────┐
│ employee_id ┆ datetime_start      ┆ datetime_end        ┆ type    ┆ shift_id ┆ duration (hrs) │
│ ---         ┆ ---                 ┆ ---                 ┆ ---     ┆ ---      ┆ ---            │
│ i64         ┆ datetime[μs]        ┆ datetime[μs]        ┆ str     ┆ i64      ┆ i64            │
╞═════════════╪═════════════════════╪═════════════════════╪═════════╪══════════╪════════════════╡
│ 1           ┆ 2023-01-02 00:00:00 ┆ 2023-01-02 05:00:00 ┆ working ┆ 1        ┆ 5              │
│ 1           ┆ 2023-01-02 06:00:00 ┆ 2023-01-02 11:00:00 ┆ working ┆ 3        ┆ 5              │
└─────────────┴─────────────────────┴─────────────────────┴─────────┴──────────┴────────────────┘


In [9]:
collapsed_df = result.groupby('employee_id').agg([
    pl.col('datetime_start').first(),
    pl.col('datetime_end').last(),
    pl.col('type').first(),
]).sort("employee_id", descending=False)
print(collapsed_df)

shape: (1, 4)
┌─────────────┬─────────────────────┬─────────────────────┬─────────┐
│ employee_id ┆ datetime_start      ┆ datetime_end        ┆ type    │
│ ---         ┆ ---                 ┆ ---                 ┆ ---     │
│ i64         ┆ datetime[μs]        ┆ datetime[μs]        ┆ str     │
╞═════════════╪═════════════════════╪═════════════════════╪═════════╡
│ 1           ┆ 2023-01-02 00:00:00 ┆ 2023-01-02 11:00:00 ┆ working │
└─────────────┴─────────────────────┴─────────────────────┴─────────┘


keep in mind and use well with the following ```with_columns``` subfunctions: ```.dt.hours()```, ```.alias("")```, ```.str.strptime(pl.Datetime```, ```fmt="%Y-%m-%d %H:%M", strict=False)```, ```.select("")```, ```.apply(lambda x: ...)```, ```.cumsum()```, ```.groupby("")``` etc

